In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from urllib.request import urlopen
import json
import os
import glob
import pandas as pd

import dash
#from utils import (create_paths_file,min_max_date,slicer,create_tree,create_map_bubble_year,create_curve_line,)



In [2]:
app = JupyterDash(__name__)

# Load data
# Load in the Data
PATH_CSV = "/Users/nourahalsaadan/Downloads/Sample - Superstore.csv"
df = pd.read_csv(PATH_CSV)

print(df.shape)
df.head()

(9994, 22)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,ZIP,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,2416,CA-2016-146087,06/07/2016,11/07/2016,Standard Class,PP-18955,Paul Prost,Home Office,United States,Holyoke,...,1040,East,FUR-BO-10002824,Furniture,Bookcases,Bush Mission Pointe Library,301.96,2,0.0,60.3920
1,5769,CA-2016-154900,25/02/2016,01/03/2016,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
2,2032,CA-2016-117961,26/11/2016,30/11/2016,Standard Class,GP-14740,Guy Phonely,Corporate,United States,Leominster,...,1453,East,OFF-PA-10001274,Office Supplies,Paper,Loose Memo Sheets,10.95,3,0.0,5.1465
3,2029,CA-2016-117961,26/11/2016,30/11/2016,Standard Class,GP-14740,Guy Phonely,Corporate,United States,Leominster,...,1453,East,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",14.73,3,0.0,6.9231
4,5770,CA-2016-154900,25/02/2016,01/03/2016,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240


In [3]:
df.describe()

,Row ID,ZIP,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.371723,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.705315,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [4]:
def run_server(self,
               port=8050,
               debug=True,
               threaded=True,
               **flask_run_options):
    self.server.run(port=port, debug=debug, **flask_run_options)

In [ ]:
# Initialise the app
app = dash.Dash(__name__)

# sales df - for map
df_sales = df.groupby(['State','State Code', 'ZIP'])[['Sales']].mean()
df_sales.reset_index(inplace=True)
# profit df - for map 
df_profit = df.groupby(['State','State Code', 'ZIP'])[['Profit']].mean()
df_profit.reset_index(inplace=True)

# segment label - for scatter  
segment_label = ['Consumer', 'Corporate', 'Home Office', 'All']

# cat label - for scatter and hist 
cat_label = ['Office Supplies','Furniture','Technology','All']

# app layout 
app.layout = html.Div([

     html.Div([
         
         # Banner display
         html.H1("Sample Superstore Dashboard", 
                 style={'textAlign':'center', 'color': '#FBFBF9', 'paddingBottom': '5px'}),
         
         # First - Map 
         html.H3("Map the sales & profit", 
                 style={'color': '#FBFBF9'}),        
         
         html.Div([
             html.Label([' Pick one to show ',
                         dcc.RadioItems(
                             id="sales_profit",
                             options=[{'label': 'Sales', 'value': 'sal'},
                                      {'label': 'Profit', 'value': 'pro'}],
                             value='sal',
                             labelStyle={'display': 'inline-block', 'padding' : '10px'})])
             ],style={'width': '100%', 'display': 'inline-block', 'color': '#FBFBF9'})
     ], style={
         'padding': '10px 5px',
         'backgroundColor':'#2B3856'
     }),
    
    html.Div([
        dcc.Graph(id='map')
    ], style={'width': '99.3%', 'display': 'inline-block', 'padding': '10px 5px', 'backgroundColor':'#2B3856'}),
    
    
    # Second - Scatter
    html.Div([
        
        html.H3("Segment and Sub category relationship", 
                style={'color': '#FBFBF9'}),
        
        html.Div([
            html.Label([' Pick the Segment',
                        dcc.RadioItems(
                            id="segment",
                            options=[{'label':x, "value":x} for x in segment_label],
                            value='All',
                            labelStyle={'display': 'inline-block', 'padding': '10px'})])
             ], style={'width': '100%', 'display': 'inline-block', 'color': '#FBFBF9'})
    ], style={
         'padding': '10px 5px',
         'backgroundColor':'#2B3856'
     }),
        
    html.Div([
        dcc.Graph(id='mram')
    ], style={'width': '99.3%', 'display': 'inline-block', 'padding': '10px 5px', 'backgroundColor':'#2B3856'}),
    
    # Third and Fourth - scatter and hist 
    html.Div([
        
        html.H3("Effects of sub category on sales and profit", 
                style={'color': '#FBFBF9'}),
        
        html.Div([
            html.Label([' Pick the Category',
                        dcc.Dropdown(
                            id="Category",
                            value='All',
                            options=[{'label':x, 'value':x} for x in cat_label],
                            style={'color': '#2B3856', 'PaddingTop':'10px'})])
        ], style={'width': '100%', 'display': 'inline-block', 'color': '#FBFBF9'})
    ], style={
        'padding': '10px 5px',
        'backgroundColor':'#2B3856'
    }),
    
    html.Div([
        dcc.Graph(id='nouf'),
    ], style={'width': '49.29%', 'display': 'inline-block', 'padding': '10px 5px','backgroundColor':'#2B3856'}),
    
    html.Div([
        dcc.Graph(id='nora')
    ], style={'width': '49.29%', 'display': 'inline-block', 'float':'right', 'padding': '10px 5px','backgroundColor':'#2B3856'})
    
]) # end of app
                
#----------------------------------------------call of first grath--------------------------------------------------
@app.callback(
    Output(component_id='map', component_property='figure'),
    Input(component_id="sales_profit", component_property= "value"),
)

def update_figure(val_sal_pro):
    if (val_sal_pro == "sal"):
        fig = px.choropleth(
            data_frame=df_sales,
            locationmode='USA-states',
            locations='State Code',
            scope="usa",
            color='Sales',
            hover_data=['State', 'Sales'],
            color_continuous_scale="rdbu",
            range_color=(0, 100),
            labels={'Sales': 'Sales'},
            template='simple_white'
        )
        return fig
    
    else: 
        fig = px.choropleth(
            data_frame=df_profit,
            locationmode='USA-states',
            locations='State Code',
            scope="usa",
            color='Profit',
            hover_data=['State', 'Profit'],
            color_continuous_scale="rdbu",
            range_color=(0, 100),
            labels={'Profit': 'Profit'},
            template='seaborn'
        )
        return fig
    
#----------------------------------------------call of second grath--------------------------------------------------
@app.callback(
    Output('mram', 'figure'),
    [Input("segment", "value")]
)
def update_figure(val_segment):
    if  (val_segment != "All"):
        test = df[df["Segment"] == val_segment]
        return px.scatter(
            test,
            x="Sub-Category", 
            y="Quantity",
            render_mode="webgl",
            facet_row="Category",
            color="Category"
        )
    # "default"
    else:
        test = df
        return px.scatter(
            test, 
            x="Sub-Category", 
            y="Quantity",
            render_mode="webgl",
            facet_row="Category",
            color="Category"
        )

    
#----------------------------------------------call of third grath--------------------------------------------------
@app.callback(
    Output(component_id='nouf', component_property='figure'),
    Input(component_id="Category", component_property= "value"),
)

def update_figure(val_Category):
    if (val_Category != 'All'):
        test1 = df[(df["Category"] == val_Category)]
        return px.scatter(test1,
                          x = "Sales", 
                          y = "Profit", 
                          labels={"x":"Sales", "y":"Profit"}, 
                          color='Category',
                          hover_data=["Category"]) 
    # default
    else:
        test1 = df
        return px.scatter(
            test1,
            x = "Sales",
            y = "Profit", 
            labels={"x":"Sales", "y":"Profit"}, 
            color='Category',
            hover_data=["Category"]) 
             

#----------------------------------------------call of four grath--------------------------------------------------    
@app.callback(
    Output(component_id='nora', component_property='figure'),
    Input(component_id="Category", component_property= "value"),
)

def update_figure(val_Category):
    
    if (val_Category != 'All'):
        test1 = df[(df["Category"] == val_Category)]
        return px.histogram(test1,
                          x = "Sub-Category", 
                          y = "Profit", 
                          labels={"x":"Sub-Category", "y":"Profit"}, 
                          color='Sub-Category',
                          hover_data=["Sub-Category"]) 
    # default
    else:
        test1 = df
        return px.histogram(test1,
                            x = "Sub-Category",
                            y = "Profit", 
                            labels={"x":"Sub-Category", "y":"Profit"}, 
                            color='Category',
                            hover_data=["Category"]) 
   

if __name__=="__main__":
    app.run_server(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 5000)))

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jun/2021 06:24:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:24:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:24:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:24:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:24:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:24:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:24:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:26:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:27:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:27:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:29:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 06:29:43] "POST /_dash-update-component H